In [1]:
# https://search.mediacloud.org/
# https://pypi.org/project/mediacloud-metadata/
# https://github.com/mediacloud/mc-providers/blob/main/mc_providers/test/test_onlinenews.py#L272

import datetime
import json
import pandas as pd
import requests
from mcmetadata import extract

import mediacloud.api

# Init the connection
mc = mediacloud.api.DirectoryApi('56196a395ee77c33a296073fa08e72f541362a10')

# Read the query.csv file to obtain information of each item to dump
df_query = pd.read_csv('query.csv')


In [2]:
size_query = df_query.shape[0]
print(size_query)

6366


In [3]:
# Create the result dataframe
df_result = pd.DataFrame(columns=['year', 'title', 'url', 'text_content'])

# Create the empty JSON file
with open('data.json', 'w') as f:
    # Write empty data
    json.dump("", f)



for counter in range(0,size_query) :
    # Query the MediaCloud API with each item
    url_data = df_query.iloc[counter].url
    try: 
        metadata = extract(url=url_data)
        metadata['publication_date'] = metadata['publication_date'].year
    
        # Append data to the result dataframe
        list_metadata = []
        for field in ['publication_date', 'article_title', 'url', 'text_content'] :
            list_metadata.append(metadata[field])
        # Append data to the result dataframe
        df_result.loc[counter] = list_metadata
        # And write data in JSON file
        with open('data.json', 'a') as f:
            json.dump(metadata, f)
            f.write('\n')
    except Exception as e:
        print(f'Erreur -> "{e}", pour l\'url : {url_data}' )
        log_message = f"{datetime.datetime.now()} - {e} ; {url_data}\n"
        with open('log.txt', 'a') as log_file:
            log_file.write(log_message)

with open('log.txt', 'a') as log_file:
    log_file.write("----------------------------------\n")
print(df_result.describe)

Erreur -> "Webpage didn't return content (404) from https://www.airforcetimes.com/intel-geoint/isr/2022/06/03/unlocking-airborne-isr-can-help-achieve-regional-security-in-indo-pacific/", pour l'url : https://www.airforcetimes.com/intel-geoint/isr/2022/06/03/unlocking-airborne-isr-can-help-achieve-regional-security-in-indo-pacific/
Erreur -> "Content is too short", pour l'url : https://www.ajc.com/news/atlanta-news/among-us-police-foundations-atlanta-is-an-outlier/QKI6RLHDUBC6TKEDHDITL53CIY/
Erreur -> "Content is too short", pour l'url : https://www.ajc.com/news/coronavirus/as-covid-19-testing-dwindles-scientists-look-at-sewage-for-answers/TCXEGKZQAVEJHJFD5BF7GESFSA/
Erreur -> "Content is too short", pour l'url : https://www.ajc.com/news/coronavirus/as-covid-19-testing-dwindles-scientists-look-at-sewage-for-answers/TCXEGKZQAVEJHJFD5BF7GESFSA/%5Bobject%20Object%5D/
Erreur -> "Content is too short", pour l'url : https://www.ajc.com/news/coronavirus/as-covid-19-testing-dwindles-scientists-

In [4]:
# Export dataframe to a feather format file
df_result.reset_index().to_feather('results_v3.feather')